In [1]:
from pynq.overlays.base import BaseOverlay
import time
from datetime import datetime
base = BaseOverlay("base.bit")
#import asyncio
import threading

In [2]:
%%microblaze base.PMODA

#include "gpio.h"
#include "pyprintf.h"

//Function to turn on/off a selected pin of PMODA
void write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
}

//Function to read the value of a selected pin of PMODB
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    return gpio_read(pin_in);
}

//Add a C++ function to reset all the GPIO pins on the chosen PMOD
void reset_gpio(){
    for (unsigned int j = 0; j <= 7; j = j + 1) { 
        write_gpio(j, 0);
        //write_gpio(pin,val)
        //we want the value reset to zero
        //12 pins, starts at 0
    }
}


In [3]:
def Change_LED(new_input, element):
    global LED_Status, user_input
    reset_gpio()
    LED_Status[element] = new_input
    user_input = False
    #print(LED_Status)
    

In [4]:
def LED():
    global LED_Status, PWM_on, PWM_off, PWM_run_time, sleep_time
    if LED_Status[0] == 0: #IF OFF
        time.sleep(sleep_time)
    else: #IF ON
        if LED_Status[3] == 3: #full bright (NO PWM)
            if LED_Status[2] == 0: #If NO PWM & Cycle Color
                for i in range(1,4): #cycle write from 1 (blue), 2 (red) and 3 (green)
                    write_gpio(i,1) #turn on  LED
                    time.sleep(sleep_time) #sleep with ON
                    write_gpio(i,0) #turn off LED
                    if LED_Status[1] == 1: #if supposed to blink, sleep
                        time.sleep(sleep_time)
            else: #IF NO PWM & Solid Color
                write_gpio(LED_Status[2],1)
                time.sleep(sleep_time)
                if LED_Status[1] == 1: #if supposed to blink, sleep
                        write_gpio(LED_Status[2],0)
                        time.sleep(sleep_time)
        else: #If NOT full brightness (PWM needed)
            if LED_Status[2] == 0: #PWM & cycle thru colors
                for i in range(1,4):
                    for j in range(PWM_run_time):
                        write_gpio(i,1) #turn on LED
                        time.sleep(PWM_on[LED_Status[3]]) #sleep with ON
                        write_gpio(i,0) #turn off LED
                        time.sleep(PWM_off[LED_Status[3]]) #sleep with OFF
                    if LED_Status[1] == 1: #If blinking
                        time.sleep(sleep_time)
            else:
                for i in range(PWM_run_time):
                    write_gpio(LED_Status[2],1) #turn on LED
                    time.sleep(PWM_on[LED_Status[3]]) #sleep with ON
                    write_gpio(LED_Status[2],0) #turn off LED
                    time.sleep(PWM_off[LED_Status[3]]) #sleep with OFF
                if LED_Status[1] == 1: #If Blinking
                    time.sleep(sleep_time)

In [5]:
def testLED():
    global LED_State, test, change_value, change_element, user_input
    if test == 3:                 #1) TURN ON
        print('Turn On (RED)')
        change_value = 1
        change_element = 0
        #LED_Status[0] = 1
        user_input = True
    elif test == 5:               #2) Blue
        print('Change Color to Blue')
        change_value = 1
        change_element = 2
        #LED_Status[2] = 1
        user_input = True
    elif test == 7:               #3) Green
        print('Change color to Green')
        change_value = 3
        change_element = 2
        #LED_Status[2] = 3
        user_input = True
    elif test == 9:               #4) Cycle thru
        print('Cycle through colors (B->R->G->B...)')
        change_value = 0
        change_element = 2
        #LED_Status[2] = 0
        user_input = True
    elif test == 11:              #5) Blink (while cycling)
        print('Start to Blink off every 1s, Still Cycle colors')
        change_value = 1
        change_element = 1
        #LED_Status[1] = 1
        user_input = True
    elif test == 13:              #6) Red (While Blinking)
        print('Color now Red only (Still blinking)')
        change_value = 2
        change_element = 2
        #LED_Status[2] = 2
        user_input = True
    elif test == 15:              #7) Brightness LOW (Red & blinking)
        print('Brightness Low (blinks every 1s & is Red)')
        change_value = 0
        change_element = 3
        LED_Status[3] = 0
        user_input = True
    elif test == 17:              #8) Cycle Colors (While low brightness & Blinking)
        print('Cycle thru colors while low brightness & blinking')
        change_value = 0
        change_element = 2
        LED_Status[2] = 0
        user_input = True
    elif test == 19:              #9) Red (While low brightness & Blinking)
        print('Color red (low bright & blink)')
        change_value = 2
        change_element = 2
        #LED_Status[2] = 2
        user_input = True
    elif test == 21:              #10) Solid (While Red and low brightness)
        print('Solid Red (while low brightness & red)')
        change_value = 0
        change_element = 1
        #LED_Status[1] = 0
        user_input = True
    elif test == 23:              #11) brightness up 1 (While Red and solid)
        print('Brightness up 1 level (still red and solid)')
        change_value = 1
        change_element = 3
        #LED_Status[1] = 0
        user_input = True
    elif test == 25:              #12) brightness up 1 (While Red and solid)
        print('Brightness up 1 level (still red and solid)')
        change_value = 2
        change_element = 3
        #LED_Status[1] = 0
        user_input = True
    elif test == 27:              #13) brightness up 1 - full bright (While Red and solid)
        print('Brightness Maximum.')
        print('Testing scenarios are concluded.')
        change_value = 3
        change_element = 3
        #LED_Status[1] = 0
        user_input = True

In [ ]:
#USE THIS CELL WITH ANOTHER PINK BOARD

#LED_Status = [0, 0, 2, 3] #default status
LED_Status = [1, 0, 2, 2] #testing status
#          = [off/on, solid/blink, cycle/blue/red/green, brightness_level]
Duty_Cycle = [0.04736614270344993, 0.217637640824031, 0.5310492251033824, 1.0] #25%, 50%, 75%, and 100% brightness.
#time_on = duty_cycle * (1/frequency) && time_off = (1-duty_cycle) * (1/frequency)
frequency = 20 #for the PWM Brightness
PWM_on = []
for i in range(4):
    PWM_on.append(Duty_Cycle[i]/frequency)
PWM_off = []
for i in range(4):
    PWM_off.append((1-Duty_Cycle[i])/frequency)
sleep_time = 1
PWM_run_time = frequency*sleep_time
user_input = False
change_value = 0
change_element = 0

reset_gpio()

def LOOP(_l, num):
    global user_input, Change_LED, change_value, change_element
    while True:
        resource_available = _l.acquire(False)
        if resource_available == True: #Note: We always want the same "worker" to have the lock or the LED may not work properly
            LED()
            if user_input == True:
                Change_LED(change_value, change_element)
            _l.release()
            #time.sleep(sleep_time-.1)
            #user_input(1)
        else:
            #INSERT FUNCTION TO SEE IF NEW MESSAGE CAME IN
            #This Function should set user_input = True
            
            print('Worker {} is searching for incoming message.'.format(num))
            time.sleep(sleep_time)

threads = []
fork = threading.Lock()
for i in range(2):
    #fork is the shared lock
    #i is the user (led number too)
    t = threading.Thread(target=LOOP, args=(fork, i))
    threads.append(t) #creates the new thread number
    t.start()

In [6]:
#USE THIS CELL WHEN TESTING ALONE

LED_Status = [0, 0, 2, 3] #default status
#LED_Status = [1, 0, 2, 3] #testing status
#          = [off/on, solid/blink, cycle/blue/red/green, brightness_level]
Duty_Cycle = [0.04736614270344993, 0.217637640824031, 0.5310492251033824, 1.0] #25%, 50%, 75%, and 100% brightness.
#time_on = duty_cycle * (1/frequency) && time_off = (1-duty_cycle) * (1/frequency)
frequency = 20 #for the PWM Brightness
PWM_on = []
for i in range(4):
    PWM_on.append(Duty_Cycle[i]/frequency)
PWM_off = []
for i in range(4):
    PWM_off.append((1-Duty_Cycle[i])/frequency)
sleep_time = 1
PWM_run_time = frequency*sleep_time
user_input = False
change_value = 0
change_element = 0

reset_gpio()
test = 0


def LOOP(_l, num):
    global user_input, Change_LED, change_value, change_element, test
    while True:
        resource_available = _l.acquire(False)
        if resource_available == True: #Worker 0 will ALWAYS have lock.
            LED()
            test = test+1
            if test >= 3:
                testLED()
            if user_input == True: 
                Change_LED(change_value, change_element)
                #NOTE: IS HERE instead of below so it does not change LED_Status[] while in the middle of a cycle.
            _l.release()
            #time.sleep(sleep_time-.25)
        else: #Worker 1 will ALWAYS be on network duty
            #INSERT FUNCTION TO SEE IF NEW MESSAGE CAME IN
            #This Function should set user_input = True
            
            print('    Worker {} is searching for incoming message.'.format(num))
            time.sleep(sleep_time)
            
        
        
threads = []
fork = threading.Lock()
for i in range(2):
    #fork is the shared lock
    #i is the user (led number too)
    t = threading.Thread(target=LOOP, args=(fork, i))
    threads.append(t) #creates the new thread number
    t.start()

    Worker 1 is searching for incoming message.
    Worker 1 is searching for incoming message.
    Worker 1 is searching for incoming message.
Turn On (RED)
    Worker 1 is searching for incoming message.
    Worker 1 is searching for incoming message.
Change Color to Blue
    Worker 1 is searching for incoming message.
    Worker 1 is searching for incoming message.
Change color to Green
    Worker 1 is searching for incoming message.
    Worker 1 is searching for incoming message.
Cycle through colors (B->R->G->B...)
    Worker 1 is searching for incoming message.
    Worker 1 is searching for incoming message.
    Worker 1 is searching for incoming message.
    Worker 1 is searching for incoming message.
    Worker 1 is searching for incoming message.
    Worker 1 is searching for incoming message.
Start to Blink off every 1s, Still Cycle colors
    Worker 1 is searching for incoming message.
    Worker 1 is searching for incoming message.
    Worker 1 is searching for incoming mes